In [37]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from centris.centris_scraper import CentrisBienParser, AsyncCentrisScraper
from centris.db_models import PlexCentrisListingDB
from centris import Session
from sqlalchemy.sql import select

from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Scraping 1 listing

In [21]:
plex_start_url="https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1"
url_bien_centris = "https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary"

In [22]:
centris_parser = CentrisBienParser(url_bien_centris)

In [32]:
scrape_date=datetime.now()
data = centris_parser.get_data(scrape_date)

In [30]:
pprint(data.model_dump())


{'adresse': "10627 - 10631, Rue D'Iberville, Montréal (Ahuntsic-Cartierville), "
            'Quartier Ahuntsic Est',
 'annee_construction': 1966,
 'centris_id': 19418151,
 'date_scrape': '2024-12-28',
 'description': None,
 'eval_municipale': 815000,
 'nombre_unites': 3,
 'prix': 980000,
 'quartier': 'Ahuntsic Cartierville',
 'revenus': 36000,
 'stationnement': 3,
 'style_batiment': None,
 'superficie_batiment': None,
 'superficie_commerce': None,
 'superficie_habitable': None,
 'superficie_terrain': 3401,
 'taxes': 5313,
 'title': 'Triplex à vendre',
 'unites': ['3 1/2', '5 1/2', '5 1/2'],
 'url': 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary',
 'utilisation': 'Résidentielle',
 'ville': 'Montreal'}


saving to DB

In [ ]:
db_entry = centris_parser.to_db_model(scrape_date)
with Session.begin() as session:
    session.add_all(
        [
            db_entry
        ]
    )

In [ ]:
# Query the database for testing
stmt = select(PlexCentrisListingDB)
objects = session.scalars(stmt).all()

In [44]:
objects[0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x11fce7770>,
 'unites': '["3 1/2", "5 1/2", "5 1/2"]',
 'utilisation': 'Résidentielle',
 'eval_municipale': 815000,
 'nombre_unites': 3,
 'style_batiment': None,
 'date_scrape': '2024-12-28',
 'url': 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary',
 'superficie_habitable': None,
 'adresse': "10627 - 10631, Rue D'Iberville, Montréal (Ahuntsic-Cartierville), Quartier Ahuntsic Est",
 'superficie_batiment': None,
 'ville': 'Montreal',
 'centris_id': 19418151,
 'superficie_commerce': None,
 'quartier': 'Ahuntsic Cartierville',
 'title': 'Triplex à vendre',
 'superficie_terrain': 3401,
 'prix': 980000,
 'revenus': 36000,
 'annee_construction': 1966,
 'stationnement': 3,
 'description': None,
 'taxes': 5313}

In [ ]:
with Session.begin() as session:
    # Get existing IDs
    existing_ids = {
        id_[0] for id_ in 
        session.query(PlexCentrisListingDB.centris_id).all()
    }

In [46]:
print(existing_ids)

{24248804, 19344934, 19418151, 15838313, 14787455}


In [1]:
from centris.main import get_existing_centris_ids
existing_ids = get_existing_centris_ids()

In [4]:
for el in existing_ids:
    print(el)
    break

18286084


In [5]:
all_urls = ['https://www.centris.ca/fr/duplex~a-vendre~montreal-ahuntsic-cartierville/11493867?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-cote-des-neiges-notre-dame-de-grace/21712528?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-lachine/23406148?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/26336120?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-ahuntsic-cartierville/28623760?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-l-ile-bizard-sainte-genevieve/19847268?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ville-marie/11160801?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-riviere-des-prairies-pointe-aux-trembles/18705723?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/21586957?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/14317746?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-villeray-saint-michel-parc-extension/11141292?view=Summary', 'https://www.centris.ca/fr/quintuplex~a-vendre~montreal-le-sud-ouest/16008600?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-saint-laurent/10360074?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ville-marie/9757519?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-lachine/10149723?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-le-plateau-mont-royal/15742000?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-riviere-des-prairies-pointe-aux-trembles/19866722?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-verdun-ile-des-soeurs/24411293?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-cote-des-neiges-notre-dame-de-grace/25881718?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-rosemont-la-petite-patrie/14415173?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-riviere-des-prairies-pointe-aux-trembles/12881479?view=Summary', 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ville-marie/19584463?view=Summary', 'https://www.centris.ca/fr/duplex~a-vendre~montreal-ahuntsic-cartierville/22353235?view=Summary', 'https://www.centris.ca/fr/quadruplex~a-vendre~montreal-ahuntsic-cartierville/16027183?view=Summary']

In [8]:
from tqdm import tqdm
from centris.centris_scraper import CentrisBienParser

In [10]:
for url in tqdm(all_urls, desc="Scraping content of each listing URL"):
    centris_parser = CentrisBienParser(url)
    print(centris_parser.centris_id)
    if centris_parser.centris_id in existing_ids:
        print(f"Skipping {centris_parser.centris_id} as it already exists in the DB")
        continue
    db_entry = centris_parser.to_db_model(scrape_date)

Scraping content of each listing URL: 100%|██████████| 24/24 [00:00<00:00, 26546.23it/s]

11493867
Skipping 11493867 as it already exists in the DB
21712528
Skipping 21712528 as it already exists in the DB
23406148
Skipping 23406148 as it already exists in the DB
26336120
Skipping 26336120 as it already exists in the DB
28623760
Skipping 28623760 as it already exists in the DB
19847268
Skipping 19847268 as it already exists in the DB
11160801
Skipping 11160801 as it already exists in the DB
18705723
Skipping 18705723 as it already exists in the DB
21586957
Skipping 21586957 as it already exists in the DB
14317746
Skipping 14317746 as it already exists in the DB
11141292
Skipping 11141292 as it already exists in the DB
16008600
Skipping 16008600 as it already exists in the DB
10360074
Skipping 10360074 as it already exists in the DB
9757519
Skipping 9757519 as it already exists in the DB
10149723
Skipping 10149723 as it already exists in the DB
15742000
Skipping 15742000 as it already exists in the DB
19866722
Skipping 19866722 as it already exists in the DB
24411293
Skippin